# Neural Network Methods 2


<div class="alert alert-block alert-warning">
  
<b>Notebook objectives:</b>
    
* Apply NNs methods to train and test generalization classification on:

    - Lasso Regularization
    - Genetic Algorithm regularizaion
    
* Compile the results of the performance metrics

In [ ]:
# !pip install pandas==1.4.0

In [ ]:
# !pip install keras

In [ ]:
# !pip install tensorflow

In [ ]:
# !pip install --upgrade tensorflow
# !pip install --upgrade tensorflow-gpu

# 1. Notebook set up

In [6]:
###### Import packages

### Data handling

import numpy as np
import pandas as pd
import datetime as dt
#from IPython.display import HTML, Image #display formatted texts
import warnings
warnings.filterwarnings('ignore')

### Plotting packages
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn') # pretty graphs
import matplotlib.ticker as mticker
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter, FuncFormatter
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


### Files to pickle
import pickle
import bz2
import _pickle as cPickle

### sampling
from sklearn.model_selection import train_test_split

### sk-learn pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

### Model selection

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier

# if xgboost throws and error uncomment and install xgboost package
# !pip install xgboost
# from xgboost import XGBClassifier
# from xgboost import plot_importance, to_graphviz
from sklearn.model_selection import train_test_split


# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer

# Cross-validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# Dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.compose import make_column_transformer

from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score, accuracy_score, precision_score, recall_score, average_precision_score, f1_score
from sklearn.inspection import partial_dependence


# NN methods
import tensorflow as tf
from tensorflow import keras 
from keras import metrics
from keras.models import Sequential 
from keras.layers import Dense, Dropout,Conv1D
from keras.utils  import np_utils 

# time progress bar
from tqdm.notebook import tqdm_notebook
from tqdm import tqdm
import time
from sklearn.preprocessing import StandardScaler

# Google path set up
pickles = "/content/drive/MyDrive/pickles/"
pickles_w1 = "/content/drive/MyDrive/pickles/w1/"
pickles_w2 = "/content/drive/MyDrive/pickles/w2/"
pickles_w3 = "/content/drive/MyDrive/pickles/w3/"
pickles_w4 = "/content/drive/MyDrive/pickles/w4/"
pickles_w5 = "/content/drive/MyDrive/pickles/w5/"
pickles_feature = "/content/drive/MyDrive/pickles/feature_importance/"
pickles_imbalance = "/content/drive/MyDrive/pickles/class_imbalance_learn/"
pickles_tuning = "/content/drive/MyDrive/pickles/tuning/"


# Path set up
path = "/project/data/"
path_w1 = "/project/data/w1/"
path_w2 = "/project/data/w2/"
path_w3 = "/project/data/w3/"
path_w4 = "/project/data/w4/"
path_w5 = "/project/data/w5/"
path_feature = "/project/data/feature_importance/"

# fixed values
seed = 2323
colors = {'c1':['blue', 'red'], 'c2': ['red', 'blue', 'grey', 'purple']}
bar_width = 0.3
bin_num = 25
size = {'small_tick': 9, 'tick': 10 , 'label': 14, 'sub_title': 16, 'main_title': 20}
fig_size = {'large': (30,25), 'small': (10, 5)}

# pandas display set up
pd.options.display.max_columns = None

# 2. Loading pre-processed X and y


In [7]:
# Loading X
pickled_data_X = bz2.BZ2File(path_feature + 'X_train_regularized_w2_1pct', 'rb')
X_lasso_train = cPickle.load(pickled_data_X)
pickled_data_X.close()
print(F"X_train shape: rows {X_lasso_train.shape[0]}, columns {X_lasso_train.shape[1]}")

X_train shape: rows 338282, columns 189


In [9]:
# Loading y
pickled_data_X = bz2.BZ2File(path_feature + 'y_train_regularized_w2_1pct', 'rb')
y_lasso_train = cPickle.load(pickled_data_X)
pickled_data_X.close()
print(F"X_train shape: rows {y_lasso_train.shape[0]}, columns 1")

X_train shape: rows 338282, columns 1


In [10]:
# Loading X
pickled_data_X = bz2.BZ2File(path_feature + 'X_test_regularized_w2_1pct', 'rb')
X_lasso_test = cPickle.load(pickled_data_X)
pickled_data_X.close()
print(F"X_train shape: rows {X_lasso_test.shape[0]}, columns {X_lasso_test.shape[1]}")

X_train shape: rows 59697, columns 189


In [11]:
# Loading y
pickled_data_X = bz2.BZ2File(path_feature + 'y_test_regularized_w2_1pct', 'rb')
y_lasso_test = cPickle.load(pickled_data_X)
pickled_data_X.close()
print(F"X_train shape: rows {y_lasso_test.shape[0]}, columns 1")

X_train shape: rows 59697, columns 1


In [13]:
# Loading X
pickled_data_X = bz2.BZ2File(path_feature + 'X_complete_w2_1pct', 'rb')
X_comp_pearson = cPickle.load(pickled_data_X)
pickled_data_X.close()
print(F"X_train shape: rows {X_comp_pearson.shape[0]}, columns {X_comp_pearson.shape[1]}")

# Loading y
pickled_data_X = bz2.BZ2File(path_feature + 'y_complete_w2_1pct', 'rb')
y_comp_pearson = cPickle.load(pickled_data_X)
pickled_data_X.close()
print(F"X_train shape: rows {y_comp_pearson.shape[0]}, columns 1")

X_train shape: rows 397979, columns 304
X_train shape: rows 397979, columns 1


# 2. Neural Networks Methods

In [ ]:
def models_function(X):

  # Model 1 
  # Input layer: Number of features
  # Hiden layer: 2*Num of features
  # Turns off 40% of neurons
  # Capa oculta: 2*Num of features
  # Turns off 40% of neurons
  # Hiden layer: Num of features
  # Turns off 40% of neurons
  # Output layer: 1 neuron
  # Hyper Parameters: Loss function -> binary cross entropy, metrics: Recall y AUC, Optimizer: Adam 
  n_columns = X.shape[1]

  model = Sequential()
  model.add(Dense(units=n_columns, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(units=n_columns*2, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(units=n_columns*2, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(units=n_columns, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(units=1, activation = 'sigmoid'))
  model.compile(loss = 'binary_crossentropy', metrics=['Recall', 'AUC'], 
                optimizer = 'adam')



  # Model 2
  # Input layer: Number of features 
  # Turns off 40% of neurons 
  # Hiden layer: Num of features
  # Turns off 40% of neurons
  # Output layer: 1 neuron
  # Characteristics: Loss function -> binary cross entropy, metrics: Recall y AUC, Optimizer: Adam 
  n_columns = X.shape[1]

  model2 = Sequential()
  model2.add(Dense(units=n_columns, activation='relu'))
  model2.add(Dropout(0.4))
  model2.add(Dense(units=n_columns, activation='relu'))
  model2.add(Dropout(0.4))
  model2.add(Dense(units=1, activation = 'sigmoid'))
  model2.compile(loss = 'binary_crossentropy', metrics=['Recall', 'AUC'], 
                optimizer = 'adam')



  # Model 3
  # Input layer: Number of features 
  # Turns off 40% of neurons 
  # Hiden layer: Num of features divided by 2  
  # Turns off 40% of neurons 
  # Hiden layer: Num of features divided by 4 
  # Turns off 40% of neurons 
  # Hiden layer: Num of features divided by 2  
  # Turns off 40% of neurons 
  # Input layer: Number of features 
  # Turns off 40% of neurons 
  # Output layer: 1 neuron
  # Characteristics: Loss function -> binary cross entropy, metrics: Recall y AUC, Optimizer: Adam 

  n_columns = X.shape[1]

  model3 = Sequential()
  model3.add(Dense(units=n_columns, activation='relu'))
  model3.add(Dropout(0.4))
  model3.add(Dense(units=int(n_columns/2), activation='relu'))
  model3.add(Dropout(0.4))
  model3.add(Dense(units=int(n_columns/4), activation='relu'))
  model3.add(Dropout(0.4))
  model3.add(Dense(units=int(n_columns/2), activation='relu'))
  model3.add(Dropout(0.4))
  model3.add(Dense(units=n_columns, activation='relu'))
  model3.add(Dropout(0.4))
  model3.add(Dense(units=1, activation = 'sigmoid'))
  model3.compile(loss = 'binary_crossentropy', metrics=['Recall', 'AUC'], 
                optimizer = 'adam')

  # Modelo 4
  # Input layer: Number of features 
  # Turns off 40% of neurons 
  # Hiden layer: Num of features divided by 2  
  # Turns off 40% of neurons 
  # Hiden layer: Num of features divided by 4 
  # Turns off 40% of neurons 
  # Hiden layer: Num of features divided by 8
  # Turns off 40% of neurons 
  # Hiden layer: Num of features divided by 4 
  # Turns off 40% of neurons 
  # Hiden layer: Num of features divided by 2  
  # Turns off 40% of neurons 
  # Input layer: Number of features  
  # Turns off 40% of neurons 
  # Output layer: 1 neuron 
  # Characteristics: Loss function -> binary cross entropy, metrics: Recall y AUC, Optimizer: Adam 
  n_columns = X.shape[1]

  model4 = Sequential()
  model4.add(Dense(units=n_columns, activation='relu'))
  model4.add(Dropout(0.4))
  model4.add(Dense(units=int(n_columns/2), activation='relu'))
  model4.add(Dropout(0.4))
  model4.add(Dense(units=int(n_columns/4), activation='relu'))
  model4.add(Dropout(0.4))
  model4.add(Dense(units=int(n_columns/8), activation='relu'))
  model4.add(Dropout(0.4))
  model4.add(Dense(units=int(n_columns/4), activation='relu'))
  model4.add(Dropout(0.4))
  model4.add(Dense(units=int(n_columns/2), activation='relu'))
  model4.add(Dropout(0.4))
  model4.add(Dense(units=n_columns, activation='relu'))
  model4.add(Dropout(0.4))
  model4.add(Dense(units=1, activation = 'sigmoid'))
  model4.compile(loss = 'binary_crossentropy', metrics=['Recall', 'AUC'], 
                optimizer = 'adam')
  return model, model2, model3, model4

In [ ]:
def Enconder_Function(df):
  print(df.columns)
  catg_cols_1 = ['device_model', 'app_url', 'city']
  ordinal_variables = df[catg_cols_1]
  df = df.drop(catg_cols_1, axis=1)
  return df2 

In [ ]:
X_test.data.shape

(2056963,)

In [ ]:
Xcomp_pearson

,Unnamed: 0,date,tod,insertion_order,line_item,device_type,device_make,device_model,os,browser,isp_carrier,creative_name,creative_size,app_url,channel_name,exchange,inventory_source,ad_position,ad_type,position_in_content,city,impressions,billable_impressions,viewable_impressions,total_conversions,post_view_conversions,day_of_week,media_cost_log,total_media_cost_log
0,0,16/04/2022,tod_1,insertion_order1,line_item1,Smart Phone,Samsung (All Models),Samsung Galaxy A42 5G (SM-A426B),Android 12.0,Google Chrome,Virgin Media,creative_name1,320x50,liverpoolecho.co.uk,Top 2000 Global Sites for Nielsen Display Ads,TripleLift,Unknown or Unpermissioned Inventory Source,Unknown,Standard,Unknown - Out-stream,Unknown,1,1,1,0,0,Saturday,-6.146949,-6.237877
1,1,16/04/2022,tod_2,insertion_order2,line_item2,Smart Phone,Apple (All Models),Apple iPhone,iOS 14.7,Safari (iPhone/iPod),Mobile Other,creative_name2,320x100,dailymail.co.uk/tvshowbiz,Top 2000 Global Sites for Nielsen Display Ads,Google Ad Manager,Unknown or Unpermissioned Inventory Source,BELOW_THE_FOLD,Standard,Unknown - Out-stream,Douglas,1,1,1,0,0,Saturday,-10.519674,-10.596635
2,2,16/04/2022,tod_3,insertion_order3,line_item3,Smart Phone,Apple (All Models),Apple iPhone,iOS 15.4,Safari (iPhone/iPod),BT,creative_name3,300x250,lincolnshirelive.co.uk,GroupM_Inclusion List,Google Ad Manager,Unknown or Unpermissioned Inventory Source,ABOVE_THE_FOLD,Standard,Unknown - Out-stream,Unknown,4,4,4,0,0,Saturday,-7.443899,-7.535115
3,3,16/04/2022,tod_4,insertion_order3,line_item4,Smart Phone,Samsung (All Models),Samsung Galaxy S20 FE 5G (SM-G781B),Android 12.0,Google Chrome,BT,creative_name2,320x100,manchestereveningnews.co.uk,Top 2000 non-US Sites for Nielsen Video Ads,Google Ad Manager,Unknown or Unpermissioned Inventory Source,BELOW_THE_FOLD,Standard,Unknown - Out-stream,Manchester,1,1,1,0,0,Saturday,-9.293722,-9.384694
4,4,16/04/2022,tod_5,insertion_order2,line_item5,Smart Phone,Samsung (All Models),Samsung Galaxy A12 (SM-A125F),Android 11.0,Google Chrome,Virgin Media,creative_name1,320x50,unilad.co.uk,Top 2000 Global Sites for Nielsen Video Ads,PubMatic,Unknown or Unpermissioned Inventory Source,ABOVE_THE_FOLD,Standard,Unknown - Out-stream,Bolton,4,4,4,0,0,Saturday,-6.940278,-7.031053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306529,397966,03/04/2022,tod_13,insertion_order3,line_item4,Smart Phone,Samsung (All Models),Samsung Galaxy S21 Ultra 5G (SM-G998B),Android 12.0,Google Chrome,Virgin Media,creative_name4,300x250,manchestereveningnews.co.uk,Top 2000 Global Sites for Nielsen Display Ads,Google Ad Manager,Unknown or Unpermissioned Inventory Source,ABOVE_THE_FOLD,Standard,Unknown - Out-stream,Cheadle Hulme,1,1,1,0,0,Sunday,-9.408791,-9.498022
306530,397971,03/04/2022,tod_19,insertion_order3,line_item3,Smart Phone,Apple (All Models),Apple iPhone,iOS 15.3,Safari (iPhone/iPod),Plusnet,creative_name3,300x250,manchestereveningnews.co.uk,Top 2000 non-US Sites for Nielsen Video Ads,Google Ad Manager,Unknown or Unpermissioned Inventory Source,ABOVE_THE_FOLD,Standard,Unknown - Out-stream,Unknown,1,1,1,0,0,Sunday,-9.293722,-9.384694
306531,397972,03/04/2022,tod_12,insertion_order3,line_item4,Smart Phone,Samsung (All Models),Samsung Galaxy S20 Ultra (SM-G988B),Android 11.0,Google Chrome,SuperCarrier in GB,creative_name3,300x250,manchestereveningnews.co.uk,Top 2000 non-US Sites for Nielsen Video Ads,Google Ad Manager,Unknown or Unpermissioned Inventory Source,ABOVE_THE_FOLD,Standard,Unknown - Out-stream,Unknown,1,1,1,0,0,Sunday,-9.304651,-9.396670
306532,397976,03/04/2022,tod_14,insertion_order3,line_item3,Smart Phone,Apple (All Models),Apple iPhone,iOS 15.1,Safari (iPhone/iPod),Mobile Other,creative_name2,320x100,dailymail.co.uk/news,Top 2000 US Sites for Nielsen Video Ads,Google Ad Manager,Unknown or Unpermissioned Inventory Source,ABOVE_THE_FOLD,Standard,Unknown - Out-stream,London,1,1,1,0,0,Sunday,-9.552831,-9.641123



## 2.1 Lasso regularization results


In [ ]:
X = pd.get_dummies(X_train_lasso)
y = y_train_lasso
models = models_function(X)
X_test_lasso = pd.get_dummies(X_test_lasso)

In [ ]:
Results1 = pd.DataFrame()
recall = []
auc = []
name= []
i = 1
for model in models:
  model.fit(X, y, batch_size = 128, epochs = 25, verbose = 1)
  pred = model.predict(X_test_lasso)
  score = model.evaluate(X_test_lasso, y_test_lasso, verbose=1)
  print('Loss Function:', score[0])
  print('Test recall:', score[1])
  print('Test auc:', score[2])
  print(model.summary())
  auc.append(score[2])
  recall.append(score[1])
  name.append('model_number_{}'.format(i))
  i = i+1


Epoch 1/25
2643/2643 [==============================] - 40s 15ms/step - loss: 0.0105 - recall: 0.0000e+00 - auc: 0.5038
Epoch 2/25
2643/2643 [==============================] - 32s 12ms/step - loss: 0.0090 - recall: 0.0000e+00 - auc: 0.5255
Epoch 3/25
2643/2643 [==============================] - 32s 12ms/step - loss: 0.0106 - recall: 0.0000e+00 - auc: 0.5104
Epoch 4/25
2643/2643 [==============================] - 34s 13ms/step - loss: 0.0103 - recall: 0.0000e+00 - auc: 0.5190
Epoch 5/25
2643/2643 [==============================] - 33s 12ms/step - loss: 0.0126 - recall: 0.0000e+00 - auc: 0.5028
Epoch 6/25
2643/2643 [==============================] - 32s 12ms/step - loss: 0.0141 - recall: 0.0000e+00 - auc: 0.5104
Epoch 7/25
2643/2643 [==============================] - 33s 12ms/step - loss: 0.0141 - recall: 0.0000e+00 - auc: 0.5127
Epoch 8/25
2643/2643 [==============================] - 33s 13ms/step - loss: 0.0118 - recall: 0.0000e+00 - auc: 0.5021
Epoch 9/25
2643/2643 [==================

In [ ]:
Results1['Modelo'] = name
Results1['AUC'] = auc 
Results1['Recall'] = recall 


In [ ]:
display(Results1)

,Modelo,AUC,Recall
0,model_number_1,0.500000,0.000000
1,model_number_2,0.502200,0.000000
2,model_number_3,0.655655,0.601010
3,model_number_4,0.698654,0.682819


In [ ]:
variables = ['total_conversions', 'device_make', 'inventory_source', 'tod', 'ad_type',
 'browser', 'os', 'insertion_order', 'creative_name', 'device_type',
 'device_make', 'inventory_source', 
 'post_view_conversions', 'total_conversions', 'viewable_impressions',
 'creative_size', 'billable_impressions', 'tod', 'device_type',
 'media_cost_log', 'impressions', 'viewable_impressions', 'creative_size',
 'post_view_conversions', 'day_of_week']



## 2.2 Genetic Algorithm Regularization


In [ ]:
X = pd.get_dummies(Xcomp_pearson[variables])
y = ycomp_pearson['user_response']
models = models_function(X)
x_train, x_test , y_train, y_test = train_test_split(X, y, test_size=0.40)


In [ ]:
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
Results2 = pd.DataFrame()
recall = []
auc = []
name= []
i = 1
for model in models:
  model.fit(x_train, y_train, batch_size = 128, epochs = 40, verbose = 1)
  pred = model.predict(x_test)
  score = model.evaluate(x_test, y_test, verbose=1)
  print('Loss Function:', score[0])
  print('Test recall:', score[1])
  print('Test auc:', score[2])
  print(model.summary())
  auc.append(score[2])
  recall.append(score[1])
  name.append('model_number_{}'.format(i))
  i = i+1



Epoch 1/40
1437/1437 [==============================] - 38s 24ms/step - loss: 0.0158 - recall: 0.0000e+00 - auc: 0.5228
Epoch 2/40
1437/1437 [==============================] - 34s 23ms/step - loss: 0.0122 - recall: 0.0000e+00 - auc: 0.5113
Epoch 3/40
1437/1437 [==============================] - 34s 24ms/step - loss: 0.0092 - recall: 0.0000e+00 - auc: 0.5524
Epoch 4/40
1437/1437 [==============================] - 34s 24ms/step - loss: 0.0091 - recall: 0.0000e+00 - auc: 0.4986
Epoch 5/40
1437/1437 [==============================] - 33s 23ms/step - loss: 0.0096 - recall: 0.0000e+00 - auc: 0.5214
Epoch 6/40
1437/1437 [==============================] - 34s 24ms/step - loss: 0.0131 - recall: 0.0000e+00 - auc: 0.5295
Epoch 7/40
1437/1437 [==============================] - 36s 25ms/step - loss: 0.0099 - recall: 0.0000e+00 - auc: 0.5220
Epoch 8/40
1437/1437 [==============================] - 33s 23ms/step - loss: 0.0092 - recall: 0.0000e+00 - auc: 0.5282
Epoch 9/40
1437/1437 [==================

In [ ]:
Results2['Modelo'] = name
Results2['AUC'] = auc
Results2['Recall'] = recall 
display(Results2)

,Modelo,AUC,Recall
0,model_number_1,0.500000,0.000000
1,model_number_2,0.523140,0.000000
2,model_number_3,0.655655,0.103774
3,model_number_4,0.500000,0.000000
